Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# ~/ray_results/PPO_SpainAI_1002/PPO_MyEnv_00243_00000_0_2021-02-10_21-31-48/

In [96]:
from data_utils import _save_pickle, _fix_df, cut_sp500, cut_oil, _read_sp500, _read_oil, get_raw_dataset, preprocess_data, _fill_dates
from data_utils import _which_assets_available, build_obs_df, _get_time_attributes 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import random
from constants import logret_cols

In [22]:
reference_df = pd.read_csv("submission/submission.csv", parse_dates = ["eod_ts"], index_col="eod_ts")

In [23]:
reference_df.head()

,allo_REU,allo_VRT,allo_EEY,allo_JTL,allo_SEH,allo_BSX,allo_OJG,allo_UEI,allo_HQU,allo_ZXW,allo_LEN,allo_YEC,allo_UYZ,allo_LWK,allo_ACY,allo_HEO,allo_FIR,allo_BGN
eod_ts,,,,,,,,,,,,,,,,,,
2020-08-18 00:00:00,0.114379,0.029099,0.013186,0.101076,0.101519,0.057257,0.027439,0.018777,0.053406,0.080812,0.020080,0.087687,0.070070,0.099456,0.029514,0.037835,0.016323,0.042084
2020-08-18 01:00:00,0.014112,0.088849,0.022711,0.026985,0.003099,0.040348,0.045789,0.121986,0.114450,0.107733,0.024035,0.043220,0.093317,0.008298,0.086308,0.065200,0.041210,0.052349
2020-08-18 02:00:00,0.069645,0.046509,0.026061,0.086234,0.045954,0.058223,0.046801,0.045134,0.121548,0.014297,0.067253,0.083320,0.043973,0.090604,0.016118,0.120136,0.016471,0.001718
2020-08-18 03:00:00,0.107072,0.035263,0.106222,0.007086,0.055552,0.042929,0.093544,0.069414,0.014073,0.037031,0.023287,0.088242,0.049342,0.074206,0.069467,0.104633,0.020961,0.001674
2020-08-18 04:00:00,0.060190,0.069420,0.027955,0.077568,0.062527,0.049295,0.003327,0.006662,0.019053,0.075851,0.075360,0.079179,0.023196,0.082323,0.090402,0.097320,0.015146,0.085227


In [2]:
assets_df = get_raw_dataset("trainTimeSeries/trainTimeSeries/TrainCandles")

100%|██████████| 96/96 [00:00<00:00, 4265.94it/s]


In [3]:
sp500 = _read_sp500("datos_series_temporales/IVE_tickbidask.txt")

In [4]:
oil = _read_oil("datos_series_temporales/OIH_adjusted.txt")

In [57]:
sp500.fillna(method="ffill", inplace=True)

In [58]:
oil.fillna(method="ffill", inplace=True)

In [78]:
def _get_below_cut(df, reference_df, prefix, last_date="2020-12-24 21:00:00"):
    
    first_date = reference_df.index[0]
    last_before = df.loc[df["date_time"] < first_date].iloc[-1]
    after_date = df["date_time"] >= first_date
    before_date = df["date_time"] <= pd.to_datetime(last_date)
    between = after_date & before_date
    df_cutted = df.loc[between]
    df_cutted = _fix_df(df_cutted, prefix)
    dates_ref = reference_df.index
    dates_remove = [idx for idx in df_cutted.index if idx not in dates_ref]
    df_cutted.drop(dates_remove, axis=0, inplace=True)
    
    df = _fill_dates(df, reference_df)
    
    df.fillna(last_before, inplace=True)
    return df_cutted

In [79]:
oil_cutted = _get_below_cut(oil, reference_df, "oil")

In [80]:
oil_cutted.head()

,oil_open,oil_high,oil_low,oil_close,oil_volume
date,,,,,
2020-08-18 09:00:00,135.563810,135.607143,135.507619,135.537619,695.904762
2020-08-18 10:00:00,134.486829,134.512927,134.445854,134.479268,685.439024
2020-08-18 11:00:00,133.897727,133.912273,133.879545,133.891591,338.727273
2020-08-18 12:00:00,132.860833,132.884444,132.845833,132.866389,277.111111
2020-08-18 13:00:00,132.715143,132.732286,132.694857,132.711143,374.400000


In [81]:
sp500_cutted = _get_below_cut(sp500, reference_df, "sp500")

In [83]:
sp500_cutted.head()

,sp500_price,sp500_bid,sp500_ask,sp500_volume
date,,,,
2020-08-18 08:00:00,115.450000,114.230000,117.380000,1600.000000
2020-08-18 09:00:00,115.913403,115.904007,115.929326,181.932584
2020-08-18 10:00:00,115.441976,115.432415,115.450093,235.975232
2020-08-18 11:00:00,115.546056,115.540312,115.553542,214.130208
2020-08-18 12:00:00,115.670738,115.663592,115.677573,261.436893


In [84]:
def _save_pickle(obj, file):
    with open(file, "wb") as f:
        pickle.dump(obj, f)

In [85]:
def _save_scaler_variable(df, col, name):
    arr = df[col].values.reshape(-1, 1)
    scaler = StandardScaler()
    scaler.fit_transform(arr)
    _save_pickle(scaler, f"scaler_{name}.pkl")
    return scaler

In [86]:
scaler_vol_oil = _save_scaler_variable(oil_cutted, "oil_volume", "vol_oil")

In [87]:
scaler_vol_sp = _save_scaler_variable(sp500_cutted, "sp500_volume", "vol_sp500")

In [88]:
sp500_processed = preprocess_data(sp500_cutted, reference_df, "sp500", scaler_vol_sp)

/home/alejandro.vaca/miniconda/envs/series_temporales/lib/python3.7/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [89]:
sp500_processed.head()

,sp500_price,sp500_bid,sp500_ask,sp500_volume,sp500_spread
date,,,,,
2020-08-18 00:00:00,NaN,NaN,NaN,NaN,NaN
2020-08-18 01:00:00,NaN,NaN,NaN,NaN,NaN
2020-08-18 02:00:00,NaN,NaN,NaN,NaN,NaN
2020-08-18 03:00:00,NaN,NaN,NaN,NaN,NaN
2020-08-18 04:00:00,NaN,NaN,NaN,NaN,NaN


In [75]:
#sp500.loc[sp500["date"] < pd.to_datetime("2020-08-18 00:00:00")]

In [48]:
reference_df.shape

(2229, 18)

In [49]:
oil_processed = preprocess_data(oil_cutted, reference_df, "oil", scaler_vol_oil)

In [50]:
oil_processed.shape

(2229, 5)

In [55]:
oil_processed.isna().sum()

oil_open      10
oil_high      10
oil_low       10
oil_close     10
oil_volume     9
dtype: int64

In [90]:
sp500_processed.fillna(method="ffill", inplace=True)
oil_processed.fillna(method="ffill", inplace=True)

In [91]:
sp500_processed.fillna(0, inplace=True)
oil_processed.fillna(0, inplace=True)

In [92]:
sp500_processed.shape

(2229, 5)

In [93]:
reference_df.shape

(2229, 18)

In [95]:
oil_processed.shape

(2229, 5)

In [97]:
totaldf = pd.merge(sp500_processed, oil_processed, on="date")

In [98]:
totaldf.head()

,sp500_price,sp500_bid,sp500_ask,sp500_volume,sp500_spread,oil_open,oil_high,oil_low,oil_close,oil_volume
date,,,,,,,,,,
2020-08-18 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-18 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-18 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-18 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-18 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
def _which_assets_available(df, assets_df, test=False, test_shape=2229):
    assets_df.fillna(method="ffill", inplace=True)
    if not test:
        values = [assets_df.iloc[i, :].notna().values.astype("int") for i in trange(assets_df.shape[0])]
    else:
        values = [[1]*assets_df.shape[1]]*test_shape
    columns = [f"{col.replace('_close', '')}_available" for col in assets_df.columns]
    df_available = pd.DataFrame(
        values,
        columns=columns,
        index=df.index
    )
    df = pd.merge(df, df_available, left_index=True, right_index=True, how="outer")
    return df

In [111]:
def build_obs_df(df, assetsdf):
    df = _get_time_attributes(df)
    df = _which_assets_available(df, assetsdf, test=True)
    return df

In [112]:
obsdf_test = build_obs_df(totaldf, assets_df)

In [113]:
obsdf_test.shape

(2229, 110)

In [114]:
obsdf_test.head()

,sp500_price,sp500_bid,sp500_ask,sp500_volume,sp500_spread,oil_open,oil_high,oil_low,oil_close,oil_volume,...,AZG_available,OJG_available,WWT_available,BOT_available,TXR_available,RAT_available,DIG_available,SRI_available,PHI_available,RWJ_available
date,,,,,,,,,,,,,,,,,,,,,
2020-08-18 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1


In [115]:
obsdf_test.columns

Index(['sp500_price', 'sp500_bid', 'sp500_ask', 'sp500_volume', 'sp500_spread',
       'oil_open', 'oil_high', 'oil_low', 'oil_close', 'oil_volume',
       ...
       'AZG_available', 'OJG_available', 'WWT_available', 'BOT_available',
       'TXR_available', 'RAT_available', 'DIG_available', 'SRI_available',
       'PHI_available', 'RWJ_available'],
      dtype='object', length=110)

In [117]:
obs_df = pd.read_csv("observations_df.csv", parse_dates=["date"], index_col="date")

In [118]:
obs_df.columns == obsdf_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [119]:
def create_synthetic_assetsdf(dates, assets_df):
    cols = assets_df.columns
    d = {col: [0]*len(dates) for col in cols}
    df = pd.DataFrame(d, index=dates)
    return df

In [120]:
synthetic_assetsdf = create_synthetic_assetsdf(obsdf_test.index, assets_df)

In [121]:
synthetic_assetsdf.shape

(2229, 96)

In [122]:
synthetic_assetsdf.to_csv("synthetic_assetsdf.csv", index=True, header=True)

In [123]:
synthetic_assetsdf.head()

,ZVQ_close,NCT_close,YAX_close,OOS_close,GFJ_close,FIR_close,USX_close,FSK_close,TMF_close,TDD_close,...,AZG_close,OJG_close,WWT_close,BOT_close,TXR_close,RAT_close,DIG_close,SRI_close,PHI_close,RWJ_close
date,,,,,,,,,,,,,,,,,,,,,
2020-08-18 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-18 01:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-18 02:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-18 03:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-08-18 04:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
obsdf_test.head()

,sp500_price,sp500_bid,sp500_ask,sp500_volume,sp500_spread,oil_open,oil_high,oil_low,oil_close,oil_volume,...,AZG_available,OJG_available,WWT_available,BOT_available,TXR_available,RAT_available,DIG_available,SRI_available,PHI_available,RWJ_available
date,,,,,,,,,,,,,,,,,,,,,
2020-08-18 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1
2020-08-18 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,1,1,1,1,1,1


In [125]:
obsdf_test.to_csv("observations_test_df.csv", header=True, index=True)